In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Iniciar random
np.random.seed(6)

# Variables globales con información para acelerar procesamiento
g_filas = g_cols   = 0    # Serán números (1.000.000 y 11 en este caso)
g_pesos = g_dpesos = None # Serán arrays de 11 elementos
g_sesgo = g_dsesgo = 0    # Serán números (coma flotante)

ITER = 10
APRN = 0.5
LMBD = 0.000002

In [2]:
# linea es un string de la forma:
# "n0, n1, n2, ..., n10, r" donde nX es una propiedad del tráfico y r es
# su clasificación (1 si es botnet, 0 si no lo es)
def formato_inicial(linea):
    aux   = [float(i) for i in linea.split(',')]
    datos = np.array(aux[:-1])
    resul = aux[-1]
    return (datos, resul)

# Devuelve un RDD de los datos del fichero fn.
# Cada registro del RDD es una tupla (X, y)
#  X: np.array de los datos
#  y: clasificación (1 = botnet, 0 = no botnet)
def readFile(fn):
    global g_filas, g_cols
    sc      = SparkContext("local[*]", "BotnetParalelaPruebas")
    todo    = sc.textFile(fn)
    ret     = todo.map(formato_inicial)
    g_filas = todo.count()
    g_cols  = len(ret.take(1)[0][0])
    return ret

t1  = time.time()
rdd = readFile("../../../datos/botnet_tot_syn_l.csv")
t2  = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 3.9155519008636475 segundos.


In [3]:
def norm_tupla(tuplaXy):
    media = np.mean(tuplaXy[0])
    stdev = np.std(tuplaXy[0])
    tnorm = (tuplaXy[0] - media)/stdev
    return (tnorm, tuplaXy[1])

# Toma por parámetro un RDD de 1.000.000 de registros.
# Cada registro tiene la forma (X, y)
# X es un array con 11 flotantes
# y es 1 ó 0
# Devuelve un RDD equivalente donde la X de cada tupla está
# reescalada a N(0, 1) (media = 0, desv. est. = 1)
def normalizar(rdd):
    return rdd.map(norm_tupla)

t1 = time.time()
datos = normalizar(rdd)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
# Haz la media de uno de los X. Tiene que ser ~0
# print(np.mean(datos.take(1)[0][0])) # 0.000000000000000010092936587501423
# La desviación estándar tiene que ser ~1
# print(np.std(datos.take(1)[0][0])) # 1.0

# 1.2345e-05 = 0.000012345

Tiempo transcurrido: 4.482269287109375e-05 segundos.


In [4]:
# fila: np.array de 11 elementos (la X de las tuplas del RDD)
def sigmoide(fila):
    global g_pesos, g_sesgo
    entrada = np.matmul(fila, g_pesos) + g_sesgo
    pred = 1 / (1 + np.exp(-entrada))
    return pred
  
# datos: RDD de 1M de regs. Cada reg: tupla (X, y)
# medias: np.array de 11 elementos (derivadas de los pesos)
def drv_pesos(datos):
    global g_filas
    sumas  = datos.map(lambda x: x[0]*(sigmoide(x[0]) - x[1])).reduce(lambda x, y: x + y)
    medias = sumas/g_filas
    return medias

# datos: RDD de 1M de regs. Cada reg: tupla (X, y)
# medias: np.array de 11 elementos (derivadas de los sesgos)
def drv_sesgo(datos):
    global g_filas
    sumas  = datos.map(lambda x: sigmoide(x[0]) - x[1]).reduce(lambda x, y: x + y)
    medias = sumas/g_filas
    return medias

def presio(datos):
    global g_filas
    parcial = datos.map(lambda x: (x[1] * np.log(sigmoide(x[0]))) + ((1-x[1]) * np.log(1 - sigmoide(x[0]))))
    suma    = parcial.reduce(lambda x, y: x + y)
    return -(1/g_filas) * suma
    
def entrenar(datos, iteraciones, aprendizaje):
    global g_filas, g_cols
    global g_pesos, g_dpesos
    global g_sesgo, g_dsesgo
    g_pesos  = np.random.random([g_cols, ])
    g_dpesos = np.zeros([g_cols, ])
    for it in range(iteraciones):
        t1 = time.time()
        g_dpesos = drv_pesos(datos)
        g_dsesgo = drv_sesgo(datos)
        
        g_pesos  = g_pesos - g_dpesos * aprendizaje
        g_sesgo  = g_sesgo - g_dsesgo * aprendizaje
        
        coste = presio(datos)
        t2 = time.time()
        print(str(it) + ": " + str(coste) + " [{}]".format(t2 - t1))
    return g_pesos, g_sesgo

pesos, sesgo = entrenar(datos, ITER, APRN)

0: 0.6803102202615465 [50.35551881790161]
1: 0.674087819807617 [52.061861515045166]
2: 0.6698118367800296 [51.13590955734253]
3: 0.6666456374088678 [52.418551206588745]
4: 0.6641917066055787 [50.4379198551178]
5: 0.6622034529410922 [50.724796295166016]
6: 0.6605219687843605 [50.78321146965027]
7: 0.6590444209905983 [50.42699122428894]
8: 0.6577041717318348 [51.000675678253174]
9: 0.6564579554903279 [50.93979620933533]
